In [1]:
import requests

In [2]:

CANVAS_API_TOKEN = ""

In [8]:
enrollments[2]['grades']

{'html_url': 'https://canvas.nau.edu/courses/29211/grades/ID',
 'current_grade': None,
 'current_score': None,
 'final_grade': 'F',
 'final_score': 0.0,
 'unposted_current_score': None,
 'unposted_current_grade': None,
 'unposted_final_score': 0.0,
 'unposted_final_grade': 'F'}

In [13]:
    course_id = '25655'  # Replace with your actual course ID
    base_url = f'https://canvas.nau.edu/api/v1/courses/{course_id}'
    headers = {
        "Authorization": f"Bearer {CANVAS_API_TOKEN}"
    }


In [14]:
    # Step 1: Get all assignments
    assignments_url = f'{base_url}/assignments'
    assignments_response = requests.get(assignments_url, headers=headers)

In [18]:
print(assignments_url)
print(assignments_response)

https://canvas.nau.edu/api/v1/courses/25655/assignments
<Response [200]>


In [20]:
    try:
        assignments_response.raise_for_status()
        assignments = assignments_response.json()
    except requests.exceptions.RequestException as e:
         print(request, 'error.html', {'message': f'Request error: {e}'})

In [31]:
print(len(assignments))
print(assignments[0]['name'])
print(assignments[1]['name'])
print(assignments[2]['name'])

3
Homework #1
TEMPLATE: Introductions
Survey | Self-evaluation 06


In [32]:
    # Step 2: Get all enrollments (students)
    enrollments_url = f'{base_url}/enrollments'
    enrollments_response = requests.get(enrollments_url, headers=headers)

In [33]:
    try:
        enrollments_response.raise_for_status()
        enrollments = enrollments_response.json()
    except requests.exceptions.RequestException as e:
        print(request, 'error.html', {'message': f'Request error: {e}'})

In [43]:
enrollments[0].keys()

dict_keys(['id', 'user_id', 'course_id', 'type', 'created_at', 'updated_at', 'associated_user_id', 'start_at', 'end_at', 'course_section_id', 'root_account_id', 'limit_privileges_to_course_section', 'enrollment_state', 'role', 'role_id', 'last_activity_at', 'last_attended_at', 'total_activity_time', 'sis_account_id', 'sis_course_id', 'course_integration_id', 'sis_section_id', 'section_integration_id', 'sis_user_id', 'html_url', 'user'])

In [46]:
for enrolment_index in range(0, len(enrollments)):
    print(enrollments[enrolment_index]["user_id"])
    print(enrollments[enrolment_index]["role"])
    if enrollments[enrolment_index]["user_id"] == 74890:
        print(enrollments[enrolment_index]["user"]["name"])


4157
DesignerEnrollment
52296
DesignerEnrollment
20988
DesignerEnrollment
64143
DesignerEnrollment
74890
StudentEnrollment
Test Student
4157
Instructor
52296
Instructor
20988
Instructor
64143
Instructor


In [84]:
    for enrollment in enrollments:
        # Filter for students only
        if enrollment.get('role') == 'StudentEnrollment':
            student_name = enrollment.get('user', {}).get('name', 'Unknown')
            user_id = enrollment.get('user', {}).get('id')
            total_grade = 0
            total_possible_points = 0
            
            for assignment in assignments:
                assignment_id = assignment.get('id')
                points_possible = assignment.get('points_possible', 0)
                
                if not assignment_id or points_possible == 0:
                    continue
                
                # Get submission for the current student for the current assignment
                submission_url = f'{base_url}/assignments/{assignment_id}/submissions/{enrollment["user_id"]}'
                submission_response = requests.get(submission_url, headers=headers)


                submission_response.raise_for_status()
                submission = submission_response.json()
                print(submission_url)
                print(submission['score'])
                #speed_grader_url = f'https://canvas.nau.edu/courses/25655/gradebook/speed_grader?assignment_id={assignment_id}&student_id={enrollment["user_id"]}'
                #speed_grader_response = requests.get(speed_grader_url, headers=headers)

https://canvas.nau.edu/api/v1/courses/25655/assignments/529887/submissions/74890
6.0
https://canvas.nau.edu/api/v1/courses/25655/assignments/679792/submissions/74890
5.0


In [82]:
submission_url

'https://canvas.nau.edu/api/v1/courses/25655/assignments/679792/submissions/74890'

In [83]:
submission['score']

5.0

In [71]:
submission_url.json()
#    print("Grade:", submission_details.get('grade'))
#    print("Score:", submission_details.get('score'))

AttributeError: 'str' object has no attribute 'json'

In [66]:
print(speed_grader_url)
print(speed_grader_response)
#https://canvas.nau.edu/courses/25655/gradebook/speed_grader?assignment_id=529887&student_id=74890

https://canvas.nau.edu/courses/25655/gradebook/speed_grader?assignment_id=679792&student_id=74890
<Response [200]>


In [69]:
speed_grader_response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [52]:
submission['score']

In [27]:
def assignment(request):







    

    student_grades = []
    total_grades_sum = 0
    total_students_count = 0

    # Step 3: Loop through each student and each assignment to get grades
    for enrollment in enrollments:
        # Filter for students only
        if enrollment.get('role') == 'StudentEnrollment':
            student_name = enrollment.get('user', {}).get('name', 'Unknown')
            user_id = enrollment.get('user', {}).get('id')
            total_grade = 0
            total_possible_points = 0
            
            for assignment in assignments:
                assignment_id = assignment.get('id')
                points_possible = assignment.get('points_possible', 0)
                
                if not assignment_id or points_possible == 0:
                    continue
                
                # Get submission for the current student for the current assignment
                submission_url = f'{base_url}/assignments/{assignment_id}/submissions/{enrollment["user_id"]}'
                submission_response = requests.get(submission_url, headers=headers)
                
                try:
                    submission_response.raise_for_status()
                    submission = submission_response.json()
                    print("========================================")
                    print(submission)
                    grade = submission.get('score', 0)  # Get the score (0 if not submitted)
                    total_grade += grade
                    total_possible_points += points_possible
                except requests.exceptions.RequestException:
                    continue
            
            # Calculate the percentage grade
            if total_possible_points > 0:
                percentage_grade = (total_grade / total_possible_points) * 100
            else:
                percentage_grade = 0
            
            student_grades.append({
                'student_name': student_name,
                'total_grade': percentage_grade,
                'user_id' : user_id
            })
            
            # Add to the total grades sum and increment the student count
            total_grades_sum += percentage_grade
            total_students_count += 1

    # Calculate the average grade
    average_grade = total_grades_sum / total_students_count if total_students_count > 0 else 0

    # Step 4: Pass the data to the template
    context = {
        'students': student_grades,
        'average_grade': average_grade,
        'total_students': total_students_count
    }

    return render(request, 'assignment.html', context)